Emotion Recognition AI Project

### 사용할 모델 다운로드

#### face-detection-adas-0001 다운로드

In [66]:
# !omz_downloader --name face-detection-adas-0001 --precision FP16

#### emotions-recognition-retail-0003 다운로드

In [67]:
# ! omz_downloader --name emotions-recognition-retail-0003 --precision FP16

## 필요한 라이브러리 imports

In [68]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

#### Inference 할 디바이스 확인

In [69]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

#### Load the Model

In [70]:
core = ov.Core()

model = core.read_model(model='./models/face-detection-adas-0001.xml')
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


In [71]:
model = core.read_model(model='./models/emotions-recognition-retail-0003.xml')
emotion_model = core.compile_model(model=model, device_name="CPU")

emotion_input_layer = emotion_model.input(0)
emotion_output_layer = emotion_model.output(0)

print("Input layer shape: ", emotion_input_layer.shape)
print("Output layer shape:", emotion_output_layer.shape)

Input layer shape:  [1,3,64,64]
Output layer shape: [1,5,1,1]


#### Load Image

In [72]:
# frame = cv2.imread("images/test.jpg")
frame = cv2.imread("images/emotions.jpg")

resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

### DrawBoundingBoxes() 함수 만들기

In [73]:
def DrawBoundingBoxes(output, frame, conf=0.5):
    boxes = []
    canvas = frame.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = (detection[3:7] * np.array([w, h, w, h])).astype("int") # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box   # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 그리기
        boxes.append(box)     #이미지에 박스를 그린 얼굴의 위치 저장
   
    return boxes

In [74]:
def DrawText(output, frame, face_position):
    # emotions 딕셔너리 생성
    emotions = {
        0:"neutral",
        1:"happy",
        2:"sad",
        3:"surprise",
        4:"anger"
    }
    # 딕셔너리 출력하기
    #for key, value in emotions.items():
    #    print(key, value, end='      ')
    #print()
        
    predictions = output[0,:,0,0]              # 5개의 감정 예측값 저장
    print("predictions : " + str(predictions))
    
    topresult_index = np.argmax(predictions)   # 5개의 감정 예측값 중 가장 높은값의 위치 저장
    print("topresult_index : " + str(topresult_index))
    
    emotion = emotions[topresult_index]        # emotions에서 topresult_index 값에 해당하는 감정 저장
    print("emotion : " + emotion)
    
    cv2.putText(frame, emotion,                 # 예측한 감정값 이미지에 출력하기
                (face_position[0],face_position[1]),    #xmin, ymin 값을 가져와 위치 설정
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 255,255), 2)

In [75]:
face_output = face_model([input_frame])[face_output_layer]
boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)

In [76]:
if boxes is not None:
    
    for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
        xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
        emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
        # 감정 인식 모델을 사용하기 위해 이미지 전처리
        # Input layer shape:  [1,3,64,64]
        resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
        transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
        input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

        emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
        DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기

predictions : [0.47297916 0.00922722 0.03208939 0.12683502 0.35886925]
topresult_index : 0
emotion : neutral
predictions : [1.8001848e-04 1.5403839e-03 9.2567629e-01 2.4386642e-04 7.2359510e-02]
topresult_index : 2
emotion : sad
predictions : [5.2245539e-03 7.2852598e-04 8.5551449e-04 9.9174678e-01 1.4446168e-03]
topresult_index : 3
emotion : surprise
predictions : [1.6682535e-01 8.2122606e-01 9.5314868e-03 1.8027051e-04 2.2368992e-03]
topresult_index : 1
emotion : happy
predictions : [3.9358577e-04 9.7996449e-01 1.5637865e-02 2.5980247e-03 1.4059979e-03]
topresult_index : 1
emotion : happy
predictions : [5.8812596e-04 5.6197669e-02 1.8172298e-04 9.1954255e-01 2.3489935e-02]
topresult_index : 3
emotion : surprise
predictions : [6.7086010e-05 9.9930334e-01 7.4545591e-05 4.5859793e-04 9.6366552e-05]
topresult_index : 1
emotion : happy
predictions : [5.3750433e-04 1.4339949e-04 1.6935905e-02 4.8260146e-05 9.8233497e-01]
topresult_index : 4
emotion : anger
predictions : [0.0036957  0.98402

In [77]:
cv2.imshow("emotion-recognition", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### Deploy

In [78]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 400
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [79]:
background = "./images/Background2.jpg"  #사용할 배경화면 경로

bg = cv2.imread(background)
deployment = AddBackground(frame, bg)

In [80]:
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [83]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./images/Background2.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]
    
    boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)
    
    if boxes is not None:
    
        for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
            xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
            emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
            # 감정 인식 모델을 사용하기 위해 이미지 전처리
            # Input layer shape:  [1,3,64,64]
            resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
            transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
            input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

            emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
            DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기
    
    deployment = AddBackground(frame, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()

predictions : [0.43999165 0.03355033 0.41629133 0.00726193 0.10290473]
topresult_index : 0
emotion : neutral
predictions : [0.51539105 0.03469002 0.33725375 0.01495302 0.09771217]
topresult_index : 0
emotion : neutral
predictions : [0.52952045 0.03273435 0.32687983 0.01412338 0.09674209]
topresult_index : 0
emotion : neutral
predictions : [0.5401707  0.05011288 0.29565868 0.01325124 0.10080655]
topresult_index : 0
emotion : neutral
predictions : [0.5021253  0.04258693 0.3462611  0.01242576 0.09660082]
topresult_index : 0
emotion : neutral
predictions : [0.5173991  0.04246558 0.34103444 0.00940965 0.0896913 ]
topresult_index : 0
emotion : neutral
predictions : [0.5411277  0.04047549 0.30885294 0.01322453 0.09631943]
topresult_index : 0
emotion : neutral
predictions : [0.57801765 0.04443212 0.27413493 0.01650906 0.08690624]
topresult_index : 0
emotion : neutral
predictions : [0.5873744  0.03981684 0.2623547  0.01949044 0.09096357]
topresult_index : 0
emotion : neutral
predictions : [0.57